In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import glob

In [2]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    image_array = np.array(image.getdata())
    try:
        image_array = image_array.reshape((im_height, im_width, 3)).astype(np.uint8)
    except:
        return None
    
    return image_array

In [3]:
starting_frame_num = 0

In [4]:
from queue import Queue
from threading import Thread
import sys
import time

class ImageReader:
    def __init__(self, image_file_list, queue_size=100):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.image_files = image_file_list
        self.total_frames = len(image_file_list)
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queue_size)

    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        time.sleep(1)
        return self

    def update(self):
        # keep looping infinitely
        frame_num = starting_frame_num
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return

            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file
                image_np = cv2.imread(self.image_files[frame_num])
                # add the frame to the queue
                self.Q.put([image_np, self.image_files[frame_num]])
                
                frame_num += 1
                
                if frame_num >= self.total_frames:
                    self.stop()
                    return
                
                

    def read(self):
        # return next frame in the queue
        return self.Q.get()

    def read_batch(self, n_frames, asarray=False):
        frames = []
        for idx in range(n_frames):
            frames.append(self.read())
        return frames


    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0

    def stop(self):
        # indicate that the thread should be stopped
        print('stopping')
        self.stopped = True
        
    def close(self):
        self.stop()

In [5]:
from queue import Queue
from threading import Thread
import sys
import time

class VideoWriter:
    def __init__(self, queue_size=100, timeout=3, frame_width=0, frame_height=0, video_out_file=None):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.queue_size = queue_size
        self.timeout = timeout
        self.stopped = False
        self.frame_width = frame_width
        self.frame_height = frame_height
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queue_size)
        self.video_out_file = video_out_file

    def start(self):
        
#         fourcc = cv2.VideoWriter_fourcc(*'mjpg')
#         self.out = cv2.VideoWriter(filename=self.video_out_file, fourcc=fourcc, fps=30, 
#                               frameSize=(frame_width, frame_height), isColor=True)
        # start a thread to read frames from the file video stream
        self.t = Thread(target=self.write, args=())
        self.t.daemon = True
        self.t.start()
        time.sleep(1)
        return self

    def write(self):
        # keep looping infinitely
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return

            # otherwise, ensure the queue has room in it
            if not self.Q.empty():
                frame, frame_file = self.Q.get()
#                 self.out.write(frame)
                file_name = os.path.basename(frame_file)
                frame_new_file = os.path.join(self.video_out_file, file_name)
                cv2.imwrite(frame_new_file, frame)
                # add the frame to the queue
            else:
                time.sleep(self.timeout)
    
    def add_frame(self, frame):
        self.Q.put(frame)
                
                

    def flush(self):
        while not self.Q.empty():
#             frame = self.Q.get()
#             self.out.write(frame)
            frame, frame_file = self.Q.get()
#                 self.out.write(frame)
            file_name = os.path.basename(frame_file)
            frame_new_file = os.path.join(self.video_out_file, file_name)
            cv2.imwrite(frame_new_file, frame)

    def stop(self):
        # indicate that the thread should be stopped
        print('stopping')
        self.stopped = True
        self.t.join()
        self.flush()
#         self.out.release()
        
    def close(self):
        self.stop()

In [6]:
image_folder = '/media/golden/72FFC6EE48B5CF39/drone_tracking/other-tracking/raw_frames/DJI_0002' 

image_files = glob.glob(image_folder + '/*.jpg')
# image_files = glob.glob(image_folder + '*')
image_files.sort(key=lambda file: int(file.split('.')[-2].split('_')[-1]))
#positions_folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/test-frame/'
positions_folder = '/media/golden/72FFC6EE48B5CF39/drone_tracking/other-tracking/processed_videos' 
positions_folder = os.path.join(positions_folder, os.path.basename(image_folder))

print(len(image_files), 'images found')

first_frame = 0
last_frame = None
label_every_n_frames = 1

use_heads = False
use_drone_movement = False
use_class_color = False

image_files = image_files[first_frame:last_frame:label_every_n_frames]
print('Using ', len(image_files), 'images')
print('demo image file:', image_files[0])


2260 images found
Using  2260 images
demo image file: /media/golden/72FFC6EE48B5CF39/drone_tracking/other-tracking/raw_frames/DJI_0002/DJI_0002_0.jpg


In [7]:
image_demo = Image.open(image_files[0])
frame_width, frame_height = image_demo.size
print('frame width: ', frame_width)
print('frame height: ', frame_height)


frame width:  3840
frame height:  2160


In [8]:
new_video_file = '/media/golden/72FFC6EE48B5CF39/drone_tracking/other-tracking/DJI_0002'
os.makedirs(new_video_file, exist_ok=True)
# new_video_file = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/long-buffalo-tracked' 
 
video_writer = VideoWriter(frame_width=frame_width, frame_height=frame_height, video_out_file=new_video_file)
video_writer.start()
        

In [9]:

#size of dot over animal in video
dot_radius = 4
dot_radius_head = 4

t = time.time()

# boxes_file = os.path.join(positions_folder, 'boxes.npy')
classes_file = os.path.join(positions_folder, 'localizations/classes.npy')
# scores_file = os.path.join(positions_folder, 'scores.npy')
positions_file = os.path.join(positions_folder, 'localizations/positions.npy')
if use_heads:
    head_positions_file = os.path.join(positions_folder, 'localizations/head-positions.npy')
# labeled_positions_file = os.path.join(positions_folder, 'labeled-positions.npy')
labeled_positions_file = os.path.join(positions_folder, 'localizations/tracks.npy')
if use_drone_movement:
    drone_movement_file = os.path.join(positions_folder, 'localizations/drone_movement.npy')

# boxes_list = np.squeeze(np.load(boxes_file))
classes_list = np.squeeze(np.load(classes_file))
# scores_list = np.squeeze(np.load(scores_file))
positions_list = np.squeeze(np.load(positions_file))
if use_heads:
    head_positions_list = np.squeeze(np.load(head_positions_file))

track_list = np.squeeze(np.load(labeled_positions_file))
if use_drone_movement:
    drone_movement_list = np.load(drone_movement_file)
    drone_movement_list = np.squeeze(drone_movement_list)

    print(len(drone_movement_list))

In [10]:


#create then initialize the image reader queue
image_reader = ImageReader(image_files, queue_size=1000)
image_reader.start()

point_trail = 150


colors = [(230, 25, 75),(255, 225, 25),(0, 130, 200),(245, 130, 48),(145, 30, 180),
          (70, 240, 240),(240, 50, 230),(210, 245, 60),(250, 190, 190),(0, 128, 128),
          (230, 190, 255), (170, 110, 40), (255, 250, 200), (128, 0, 0), (170, 255, 195),
          (128, 128, 0), (255, 215, 180), (0, 0, 128), (128, 128, 128), (255, 255, 255),
          (0, 0, 0), (60, 180, 75)] 
          
    
    



for frame_num, positions in enumerate(positions_list):
    frame_num = frame_num + int(first_frame / label_every_n_frames)
    if frame_num % 200 == 0:
        print(frame_num, ' processed')
    image_np, image_file = image_reader.read()
    if image_np is None:
        if self.verbose:
            print('image failed to load: \n' , image_path)
        continue
        
    mod_point_trail = point_trail
    if frame_num < point_trail:
        mod_point_trail = frame_num
    if use_drone_movement:
        drone_comp = np.cumsum(
            np.array(drone_movement_list[frame_num+1:frame_num-mod_point_trail:-1]), 0)
        
    for track_index, track in enumerate(track_list):
            
        
        if track['first_frame'] > frame_num:
            continue
        track_step = frame_num - track['first_frame']
        
        #track is over for this step
        if track_step > track['track'].shape[0]:
            continue
        
        local_point_trail = point_trail
        if local_point_trail > track_step:
            local_point_trail = track_step
        
        position = track['track'][track_step - local_point_trail:track_step]
        
        track_class = track['class']
        
        if track_class == 0:
            color = (255,0,255)
        elif track_class == 1:
            color = (0, 0, 255)
        elif track_class == 2:
            color = (255, 255, 0)
        elif track_class == 3:
            color = (0, 102, 204)
        elif track_class == 4:
            color = (0, 255, 255)
        else:
            color = (0, 0, 0)
        if not use_class_color:
            color = (0, 0, 255)
        for step in range(position.shape[0]):
            try:
                if use_drone_movement:
                    cv2.circle(image_np, (int(position[step][1] + drone_comp[position.shape[0] - step - 1][0]), 
                                          (frame_height - (int(position[step][0] - drone_comp[position.shape[0] - step - 1][1])))),
                               dot_radius, colors[track_index%len(colors)], -1)
                else:
                    cv2.circle(image_np, (int(position[step][1]), 
                                          (frame_height - (int(position[step][0])))),
                               dot_radius, colors[track_index%len(colors)], -1)

#                 cv2.circle(image_np, (int(position[step][1]), 
#                                       (frame_height - (int(position[step][0])))),
#                            dot_radius, colors[track_index%len(colors)], -1)
                
            except Exception as e:
                print(e)
                print(position.shape[0] - step - 1)
                print(position.shape[0])
                print('frame: ', frame_num, 'error drawing track point')
            
#         try:
#             cv2.circle(image_np, (int(track['head'][track_step][1]), 
#                                   (frame_height - int(track['head'][track_step][0]))), dot_radius_head, color, -1)
#         except:
#             print('frame: ', frame_num, 'error drawing head')
#             pass
        if use_heads:
            try:
                head_vector = track['head'][track_step] - track['track'][track_step]

                vec_mag = np.sqrt(head_vector[0] ** 2 +  head_vector[1] ** 2)
                head_vector = head_vector / vec_mag

                point0 = (int(track['head'][track_step][1]), (frame_height - int(track['head'][track_step][0])))
                point1 = (int(track['head'][track_step][1] + 50 *head_vector[1]), 
                          (frame_height - int(track['head'][track_step][0] + 50*head_vector[0])))

                cv2.line(image_np, point0, point1, color, 4, cv2.LINE_AA)
            except:
                print('frame: ', frame_num, 'error drawing head vector')
                continue
        
#     for pos_index, position in enumerate(positions):
        
#         if classes_list[frame_num][pos_index] == 0:
#             color = (0,0,255)
#         elif classes_list[frame_num][pos_index] == 1:
#             color = (0, 255, 0)
#         elif classes_list[frame_num][pos_index] == 2:
#             color = (255, 0, 0)
#         else:
#             color = (100, 100, 100)
#         cv2.circle(image_np, (int(position[1]), (frame_height - int(position[0]))), dot_radius, color, -1)
#         cv2.circle(image_np, (int(head_positions_list[frame_num][pos_index][1]), 
#                               (frame_height - int(head_positions_list[frame_num][pos_index][0]))), dot_radius_head, (0,140,255), -1)

    #Change the file names so that they are properly numerically labeled every though may not be processing
    #every frame
    head, tail = os.path.split(image_file)
    tail = tail.split('_')
    new_tail = ''
    for tail_part in tail[:-1]:
        new_tail = new_tail + tail_part + '_'
    new_tail = new_tail + ('{0:04d}.jpg').format(frame_num)
    new_image_file = os.path.join(head, new_tail)
    video_writer.add_frame([image_np, new_image_file])
print('done')
video_writer.stop()   
cv2.destroyAllWindows()  
print('time: ', time.time() - t)

0  processed
200  processed
400  processed
600  processed
800  processed
1000  processed
1200  processed
1400  processed
1600  processed
1800  processed
stopping
2000  processed
2200  processed
done
stopping
time:  255.84150624275208


To create a video, run the following script in the images folder:

ffmpeg -famerate 30 -i image-name%04d.jpg video-name.mp4

In [14]:
video_writer.stop()
cv2.destroyAllWindows()

stopping


AttributeError: 'VideoWriter' object has no attribute 't'

In [9]:
track_list[0]['track'].shape[0]

7189

In [20]:
for position in positions:
    print((frame_height - int(position[0])), int(position[1]))
print('imshape', image_np.shape)

1769 3372
1678 3207
1407 2637
1455 3002
1653 2925
1002 2523
1082 2065
958 2308
1423 2804
841 2168
1132 2427
1048 2177
1134 2559
949 2193
935 2193
imshape (2160, 4096, 3)


In [3]:
from ipywidgets import widgets
from IPython.display import display
def f(x):
    print(x)
widgets.interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-interact',))

<function __main__.f>

In [26]:
drone_move = [[1,0],[1,1],[-1,0],[0,-1],[-1,1],[1,1],[-1,-1],[0,0],[0,0]]
print(len(drone_move[1]))

2


In [55]:
frame_num = len(drone_move)
mod_point_trail = len(drone_move)
position = np.ones((9,2))
step = 5
# drone_comp = np.cumsum(np.array(drone_move[frame_num-5:frame_num-mod_point_trail:-1]), 0)
a = np.array(drone_move[10:0:-1])
print(a)
print(a.shape)
drone_comp = np.cumsum(a, 0)

# (int(position[step][1]) + drone_comp[position.shape[0] - step][1],
#  (frame_height - (int(position[step][0]) + drone_comp[position.shape[0] - step][0])))



[[ 0  0]
 [ 0  0]
 [-1 -1]
 [ 1  1]
 [-1  1]
 [ 0 -1]
 [-1  0]
 [ 1  1]]
(8, 2)


In [56]:
drone_comp

array([[ 0,  0],
       [ 0,  0],
       [-1, -1],
       [ 0,  0],
       [-1,  1],
       [-1,  0],
       [-2,  0],
       [-1,  1]])

In [11]:
import os
import shutil
folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/labeled_zebra_new' 
new_folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/zebra_labeled' 
files = os.listdir(folder)
print(os.path.join(folder, files[0]))

/media/golden/72FFC6EE48B5CF39/kenya-tracking/labeled_zebra_new/NOV08_2017_DJI_0117_0.jpg


In [12]:
num = []
error = 0
for file in files:
    try:
        newfile = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/real_this_time/NOV08_2017_DJI_' + str(int(int(file.split('.')[0].split('_')[-1])/2)) + '.jpg'
        shutil.copyfile(os.path.join(folder, file), newfile)
    except Exception as e:
        error += 1
        print(e)
        continue
print(error)

0


In [53]:
import os

value = 70000 
digits = 2
image_file = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/labeled_zebra_new_2/NOV08_2017_DJI_0117_7352.jpg'
head, tail = os.path.split(image_file)
tail = tail.split('_')
new_tail = ''
for tail_part in tail[:-1]:
    new_tail = new_tail + tail_part + '_'
new_tail = new_tail + ('{0:04d}.jpg').format(value)
new_file = os.path.join(head, new_tail)

print(new_file)
print(image_file)

/media/golden/72FFC6EE48B5CF39/kenya-tracking/labeled_zebra_new_2/NOV08_2017_DJI_0117_70000.jpg
/media/golden/72FFC6EE48B5CF39/kenya-tracking/labeled_zebra_new_2/NOV08_2017_DJI_0117_7352.jpg


['NOV08_2017_DJI_0.jpg',
 'NOV08_2017_DJI_5.jpg',
 'NOV08_2017_DJI_50.jpg',
 'NOV08_2017_DJI_500.jpg',
 'NOV08_2017_DJI_5000.jpg',
 'NOV08_2017_DJI_5001.jpg',
 'NOV08_2017_DJI_5002.jpg',
 'NOV08_2017_DJI_5003.jpg',
 'NOV08_2017_DJI_5004.jpg',
 'NOV08_2017_DJI_5005.jpg']